In [ ]:
ls -lh *_1.fq.gz | cut  -d " " -f10 > read_name_1.txt
ls -lh *_2.fq.gz | cut  -d " " -f10 > read_name_2.txt
grep "map04020" shibanyu.Gmo.expan.gene.difkegg.filt.xls | awk -F "\t" '{print$10}'|sed "s/ /\n/g"|for i in $(cat -);do grep $i shibanyu.Gmo.expan.gene.anno.xls;done > result.xls

In [ ]:
#mapping
hisat2-build -p 22 ~/genome-Ple/genome.fa Ple-genome
hisat2 -t -p 20 -x ~/thesis/02mapping/index__hisat2/Ple-genome --dta -1 ~/thesis/00all_read/trimmomatic/C_PL_L1b_1P.fq.gz -2 ~/thesis/00all_read/trimmomatic/C_PL_L1b_2P.fq.gz -S C_PL_L1b.sam
samtools sort C_PL_L1a.sam  -o C_PL_L1a.bam

#transcription
conda install -c bioconda stringtie
stringtie C_PL_L1a.bam -p 20 -o C_PL_L1a.gtf
for i in $(ls ./samfile/|sed "s/.sam//g");do echo "nohup samtools sort ./samfile/$i.sam -o $i.bam &" | bash ;done
ls ./*.gtf|sed "s/ /\n/" > gtffile.list
stringtie --merge -p22 -o merge.gtf gtffile.list
gtf_genome_to_cdna_fasta.pl merge.gtf ~/genome-Ple/genome.fa > merge.fasta
gtf_to_alignment_gff3.pl merge.gtf > merge.gff3
TransDecoder.LongOrfs -t merge.fasta 
TransDecoder.Predict -t merge.fasta
cdna_alignment_orf_to_genome_orf.pl merge.fasta.transdecoder.gff3  merge.gff3 merge.fasta > transcripts.gff3

#de nove
augustus --species=help
augustus --species=zebrafish --gff3=on --start=off --stop=off --stopCodonExcludedFromCDS=false ~/genome-Ple/genome.fa--outfile=augustus.gff3

#homology
tblastn -query refprots.fa -db target_genome.fa -outfmt 6  -num_threads 22 -max_target_seqs 10 -evalue 1e-5 > tblastn_output.tab

#EVidenceModeler

~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/gff3_gene_prediction_file_validator.pl gene.gff #验证gff可用
nohup ~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/partition_EVM_inputs.pl --genome genome.fasta --gene_predictions gene.gff --segmentSize 100000 --overlapSize 10000 --partition_listing partitions_list.out > partition.log 2>&1 &
~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/write_EVM_commands.pl --genome genome.fasta --weights /home/guisen/weights.txt --gene_predictions gene.gff --output_file_name evm.out  --partitions partitions_list.out >  commands.list
nohup ~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/execute_EVM_commands.pl commands.list >> ./execule.log 2>&1 &
~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/recombine_EVM_partial_outputs.pl  --partitions partitions_list.out --output_file_name evm.out
nohup ~/biosoftware/miniconda/opt/evidencemodeler-1.1.1/EvmUtils/convert_EVM_outputs_to_GFF3.pl --partitions partitions_list.out --output evm.out --genome ~/genome-Ple/genome.fa &
find . -regex ".*evm.out.gff3" -exec cat {} \; > EVM.all.gff3

gffread EVM.all.gff -g input/genome.fa -y tr_cds.fa
bioawk -c fastx '$seq < 50 {print $comment}' tr_cds.fa | cut -d '=' -f 2 > short_aa_gene_list.txt
grep -v -w -f short_aa_gene_list.txt EvM.all.gff > filter.gff

"""
de nove : ABINITIO_PREDICTION      AUGUSTUS  3
homology : OTHER_PREDICTION      genBlastG       7
transcription : OTHER_PREDICTION        transdecoder    10
"""

nohup /home/guisen/biosoftware/miniconda/share/transposonPSI/transposonPSI.pl cds.fa  nuc > denove.log 2>&1 &

In [ ]:
conda activate guisen
export PERL5LIB=/public/home/xuyue/anaconda3/envs/guisen/bin/perl5.26.2usr/lib/ >> ~/.bashrc

bsub -n 5 -o partition.log ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/partition_EVM_inputs.pl --genome ~/guisen/genome_file/Ple-genome.fa --gene_predictions all.gff --segmentSize 100000 --overlapSize 10000 --partition_listing partitions_list.out
bsub -n 5 -o commands.list  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/write_EVM_commands.pl --genome ../genome_file/Ple-genome.fa  --weights /public/home/xuyue/guisen/evidencemodeler/weights.txt --gene_predictions all.gff --output_file_name evm.out  --partitions partitions_list.out
bsub -n 5 -o execule.log  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/execute_EVM_commands.pl commands.list
bsub -n 5 -o recombine.log  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/recombine_EVM_partial_outputs.pl  --partitions partitions_list.out --output_file_name evm.out
bsub -n 5 -o convert.log  ~/anaconda3/envs/guisen/opt/evidencemodeler-1.1.1/EvmUtils/convert_EVM_outputs_to_GFF3.pl --partitions partitions_list.out --output evm.out --genome ../genome_file/Ple-genome.fa
find . -regex ".*evm.out.gff3" -exec cat {} \; > EVM.all.gff3
bioawk -c fastx 'length($seq) < 300 {print $name}' cds.fa > short_aa_gene_list.txt
grep -v -w -f short_aa_gene_list.txt EvM.all.gff > filter.gff
bsub -n 5 -o run.log /public/home/xuyue/anaconda3/share/transposonPSI/transposonPSI.pl cds.fa nuc

#busco
conda install -c bioconda -c conda-forge busco=5.0.0
busco -i ~/pep.fa -o busco -l actinopterygii_odb10  -m prot -c 20
generate_plot.py -wd ./

#拆分nr数据库
conda install -c bioconda taxonkit
conda install -c bioconda csvtk 
wget -b -c https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz
wget -b -c https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
taxonkit list -j 20 --ids 2759 --indent "" --data-dir ./ > Eukaryota_list
zcat prot.accession2taxid.gz |csvtk -t grep -f taxid -P Eukaryota_list |csvtk  -t cut -f accession.version > Eukaryotacc.txt
blastdbcmd -db nr_Eukaryota -entry all -dbtype prot -out nr_Eukaryota.fa
makeblastdb -in nr_Eukaryota.fa -dbtype prot -title nr_Eukaryota -parse_seqids -out nr_Eukaryota
blastp -db ~/database/blast/taxid/nr_Eukaryota  -query test.fa -evalue 1e-5 -num_threads 20 -max_target_seqs 1 -out blast.xls -outfmt 6
bsub -n 60 -q fat -o run.log blastp -db ~/guisen/database/nr_Eukaryota/nr_Eukaryota  -query ~/guisen/blast_result/pep.Ple.fa  -evalue 1e-5 -num_threads 60 -max_target_seqs 1 -out blast.xls -outfmt 6
sed -i 's/>//;s/ >/\n/g;s/\s/\t/' nr_Eukaryota.xls

#爬取kegg数据库
wget -c -b http://rest.kegg.jp/list/ko
cat ../all.fish.genes.list|cut -f1|while read id; do echo "nohup wget -c http://rest.kegg.jp/get/$id/aaseq -O $id  >> nohup.log 2>&1 & "|bash; done
bsub -n 60 -q fat -o run.log blastp -db kegg-fish/kegg_fish -query ~/guisen/blast_result/pep.Ple.fa  -evalue 1e-5 -num_threads 20 -max_target_seqs 1 -out kegg.xls -outfmt 6

#interproscan注释
nohup interproscan.sh -i ../pep.Ple.fa  -iprlookup -goterms  -dp -T temp  -f tsv -cpu 20 -o interpro.xls &

#expression
for i in $(cat ./sample.list);do bsub -c 10 -o Ple.log stringtie -e -B -p 10 -G ~/guisen/gff_file/Ple.gff3 -o ./Ple/${i}/${i}.gtf ./${i}.bam;done

#readcount
bsub -q fat -c 30 -o run.log samtools sort -n ../../C_PL_L1a.bam -o C_PL_L1a.nsort.bam
htseq-count -f bam -r name -s no -a 10 -t gene -i ID -m union sort_sam_name/C_PL_L1a.nsort.bam  ~/guisen/gff_file/Ple.gff3 > C_PL_L1a.count.xls


In [ ]:
setwd('')
getwd()
dat <- read.csv('C_PL_La_vs_PL_La.csv',row.names=1,header=TRUE)
coldata <- data.frame(row.names=colnames(dat),condition = factor(rep(c('control', 'treat'), each = 3)))
dds <- DESeqDataSetFromMatrix(countData = dat, colData = coldata, design= ~condition)
dds <- DESeq(dds)
resultsNames(dds)
res <- results(dds, contrast = c('condition', 'treat', 'control'))
summary(res)
table(res$padj<0.05)
res <- res[order(res$padj),]
diff_gene_deseq2 <-subset(res,padj < 0.05 & (log2FoldChange > 1 | log2FoldChange < -1))
diff_gene_deseq2 <- row.names(diff_gene_deseq2)
resdata <-  merge(as.data.frame(res),as.data.frame(counts(dds,normalize=TRUE)),by="row.names",sort=FALSE)
write.table(resdata,file= "result.xls",row.names = F)